<a href="https://colab.research.google.com/github/vipulgote1999/machine-learning-udacity/blob/master/UC_Merced_Land_Use_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tf-nightly-gpu
!pip install -u tensorflow_dataset 
!pip install "tensorflow_hub==0.4.0"

In [0]:
import tensorflow_datasets as tfds
import numpy as np 
import tensorflow as tf
tf.enable_eager_execution()
from __future__ import absolute_import,print_function,unicode_literals,division

import matplotlib.pylab as plt 
import tensorflow_hub as hub

from tensorflow.keras import layers


In [0]:
import logging
logger=tf.get_logger()
logger.setLevel(logging.ERROR)

In [0]:
 splits = tfds.Split.TRAIN.subsplit([80,20])

(training_set, validation_set), dataset_info = tfds.load('uc_merced', with_info=True, as_supervised=True, split=splits)

In [0]:
num_classes=dataset_info.features['label'].num_classes

num_training_examples=0
num_validation_examples=0

for examples in training_set:
  num_training_examples+=1

for examples in validation_set:
  num_validation_examples+=1
print('Total Number of Classes: {}'.format(num_classes))
print('Total Number of Training Images: {}'.format(num_training_examples))
print('Total Number of Validation Images: {} \n'.format(num_validation_examples))

In [0]:
for i, examples in enumerate(training_set.take(5)):
  print("image{} shape:{} lable:{}".format(i+1,examples[0].shape,examples[1]))

In [0]:
IMG_RES=224

def format_image(image,lable):
  image=tf.image.resize(image,(IMG_RES,IMG_RES))
  return image,lable

BATCH_SIZE=32

train_batches=training_set.shuffle(num_training_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)

validation_batch=validation_set.map(format_image).batch(BATCH_SIZE).prefetch(1)


In [0]:
url="https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

feature_extractor=hub.KerasLayer(url,input_shape=(IMG_RES, IMG_RES, 3))

In [0]:
model = tf.keras.Sequential([
  feature_extractor,
  layers.Dense(num_classes, activation='softmax')
])

model.summary()

In [0]:
model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

EPOCHS=6

history=model.fit(train_batches,epochs=EPOCHS,validation_data=validation_batch)

In [0]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [0]:
class_names = np.array(dataset_info.features['label'].names)

print(class_names)


In [0]:
image_batch, label_batch = next(iter(train_batches))


image_batch = image_batch.numpy()
label_batch = label_batch.numpy()

predicted_batch = model.predict(image_batch)
predicted_batch = tf.squeeze(predicted_batch).numpy()

predicted_ids = np.argmax(predicted_batch, axis=-1)
predicted_class_names = class_names[predicted_ids]

print(predicted_class_names)

In [0]:
print("Labels:           ", label_batch)
print("Predicted labels: ", predicted_ids)

In [0]:
plt.figure(figsize=(10,9))
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.subplots_adjust(hspace = 0.3)
  plt.imshow(image_batch[n])
  color = "blue" if predicted_ids[n] == label_batch[n] else "red"
  plt.title(predicted_class_names[n].title(), color=color)
  plt.axis('off')
_ = plt.suptitle("Model predictions (blue: correct, red: incorrect)")